# Series compartments and the effect of latency
In this notebook, we look at the effect on a simple SIR/SEIR model of
including a single or two sequential compartments representing the latent period
following infection, but before the onset of infectiousness.
We then consider the more general effect of chaining multiple compartments together in series.

## Latent period versus incubation period
The latent (or pre-infectious) period is the time from infection
to the onset of infectiousness,
whereas the incubation period is the time from infection to symptom onset.
In the simple SIR model we introduced in notebook 01,
there is no delay between infection and infectiousness,
such that the latent period is zero.
Symptoms are not explicitly represented in this model,
so the incubation period is not explicitly represented in this model
(and model structure to represent symptom status may only be necessary
if symptoms lead to some epidemiological change, 
such as through case isolation).

The serial interval and generation time are strongly influenced by these
periods, as well as by the duration of the infectious period.
The serial interval is defined as the duration of time from
the point of onset of symptoms of one infectious individual 
in a transmission chain
and the point of onset of symptoms in a person they infect.
Similarly, the generation time is defined as the duration of time 
from the infection of one individual in a transmission chain
and the point of infection in an person they infect.

## Effect on epidemic dynamics
Whether to include an intervening latency compartment between infection
and infectiousness can have significant effects on epidemic dynamics.
Perhaps the most important effect is that it will delay the initial take off of the epidemic,
although it will have only marginal effects on the final size of the epidemic.
Increasing additional latency compartments chained in series
has a less dramatic effect on epidemic dynamics,
but can better recreate the true delay from infection to infectiousness
and could be important to certain questions.
(We explore this later in this notebook.)
Therefore, whether to include one or more latency compartments depends
strongly on the epidemiological question being considered.

In [ ]:
# If running on Google Colab, run the following line of code to install the summer package
# %pip install summerepi2

In [ ]:
import pandas as pd
pd.options.plotting.backend = "plotly"

from summer2 import CompartmentalModel
from summer2.parameters import Parameter

In [ ]:
def get_sir_base_structure(
    model_config: dict,
    extra_comps=[],
) -> CompartmentalModel:
    """
    Generate a mode that doesn't do much in itself, but has some basic
    characteristics that we can then use to add in the latency structures
    that we want later on.
    We don't apply the infection process yet, as the destination compartment
    for infection will be determined later.
    
    Args:
        model_config: The fixed values used in creating the model structure
        extra_comps: Any compartments to incorporate in addition to the base ones
    Returns:
        The summer model object
    """
    
    # Compartments are comprised of the base ones and any additional latency compartments requested
    compartments = [
        "susceptible",
        "infectious",
        "recovered",
    ] + extra_comps
    
    # Otherwise the model is very similar to that from notebook 01, as follows
    infectious_compartment = ["infectious"]
    analysis_times = (
        model_config["start_time"], 
        model_config["end_time"],
    )
    model = CompartmentalModel(
        times=analysis_times,
        compartments=compartments,
        infectious_compartments=infectious_compartment,
    )
    pop = model_config["population"]
    seed = model_config["seed"]
    suscept_pop = pop - seed
    msg = "Seed larger than population"
    assert pop >= 0., msg
    model.set_initial_population(
        distribution={
            "susceptible": suscept_pop, 
            "infectious": seed}
    )
    model.add_transition_flow(
        name="recovery", 
        fractional_rate=Parameter("recovery_rate"), 
        source="infectious", 
        dest="recovered",
    )
    return model

In [ ]:
config = {
    "population": 1.,
    "seed": 0.001,
    "start_time": 0.,
    "end_time": 50.,
}

parameters = {
    "contact_rate": 1.,
    "recovery_rate": 0.333,
    "death_rate": 0.05,
    "progression": 1.,  # Progression from exposed to infectious, not used in this model configuration
}

sir_model = get_sir_base_structure(config)

sir_model.add_infection_frequency_flow(
    name="infection", 
    contact_rate=Parameter("contact_rate"), 
    source="susceptible", 
    dest="infectious",
)

sir_model.run(parameters=parameters)
sir_values = sir_model.get_outputs_df()

Let's add an exposed compartment to our based model to create the following model structure:
![](../images/seir_structure.svg)

In [ ]:
# Rebuild the model with a latent compartment
seir_model = get_sir_base_structure(
    config, 
    extra_comps=["exposed"]
)
seir_model.add_infection_frequency_flow(
    name="infection", 
    contact_rate=Parameter("contact_rate"), 
    source="susceptible", 
    dest="exposed",
)
seir_model.add_transition_flow(
    name="progression", 
    fractional_rate=Parameter("progression"), 
    source="exposed",
    dest="infectious",
)

# Run and get outputs
seir_model.run(parameters=parameters)
seir_values = seir_model.get_outputs_df()

Now let's try with two latent compartments in series:
![](../images/seeir_structure.svg)

In [ ]:
# Rebuild the model with two sequential latent compartments
extra_comps = ["exposed_0", "exposed_1"]

seeir_model = get_sir_base_structure(
    config, 
    extra_comps=extra_comps,
)
seeir_model.add_infection_frequency_flow(
    name="infection", 
    contact_rate=Parameter("contact_rate"), 
    source="susceptible", 
    dest="exposed_0",
)
seeir_model.add_transition_flow(
    name="progression_0", 
    fractional_rate=Parameter("progression") * len(extra_comps), 
    source="exposed_0",
    dest="exposed_1",
)
seeir_model.add_transition_flow(
    name="progression_1", 
    fractional_rate=Parameter("progression") * len(extra_comps), 
    source="exposed_1",
    dest="infectious",
)

# Run and get outputs
seeir_model.run(parameters=parameters)
seeir_values = seeir_model.get_outputs_df()

Having built and run these three variations on the SIR framework,
let's see how the behaviour differs between the three alternatives.

In [ ]:
results = pd.DataFrame(
    {
        "sir": sir_values["infectious"],
        "seir": seir_values["infectious"],
        "seeir": seeir_values["infectious"],
    }
)
results.plot(
    title="Epidemic trajectory and peak timing are quite different if latency is included"
)

In [ ]:
results = pd.DataFrame(
    {
        "sir": sir_values["recovered"],
        "seir": seir_values["recovered"],
        "seeir": seeir_values["recovered"],
    }
)
results.plot(
    title="Epidemic final size is very similar",
)

## Multi-compartment model
Next, let's consider the more general effects of chaining
multiple model compartments together in series.
First let's build a really simple model without any transmission dynamics at all.
The model just starts everyone out in the first compartment in the series.
Then people transition sequentially through a set of compartments with
sequential naming.
In order to make the average total duration in all the intermediate compartments
equal for all these model configurations,
we have to multiply the rate of transition between any two compartments
by the number of transitions that need to be made to reach the final compartment.
We'll arbitrarily use an average duration to arriving in the final compartment
of 30 time units (which can be thought of as days).
We'll then specify a list of different numbers of sequential compartments
we'd like to try out.

In [ ]:
def get_series_comps_model(
    n_comps: int,
) -> CompartmentalModel:
    
    # Create the base model with some simple sequential compartment names
    comps = [f"comp_{i_comp}" for i_comp in range(n_comps)]
    model = CompartmentalModel(
        times=(0, 60),
        compartments=comps,
        infectious_compartments=[],
    )
    
    # Start everyone from the first compartment in the series
    model.set_initial_population(
        distribution={"comp_0": 1.}
    )
    
    # Adjust the transition rate for the multiple compartments
    progression_rate = 1. / Parameter("sojourn_time") * (n_comps - 1)
    
    # Join up all the sequential compartments with transition flows
    for i_comp in range(n_comps - 1):
        model.add_transition_flow(
            f"progression_{i_comp}", 
            fractional_rate=progression_rate, 
            source=f"comp_{i_comp}", 
            dest=f"comp_{i_comp + 1}"
        )
    
    return model

In [ ]:
# Specify some characteristics/parameters for this very simple model - can be modified here
parameters = {"sojourn_time": 30.}
n_comp_requests = [2, 3, 4] + list(range(6, 10, 2)) + list(range(10, 105, 5))

In [ ]:
# Run the model with various numbers of sequential compartments
outputs = {}
for comp_request in n_comp_requests:
    transition_model = get_series_comps_model(comp_request)
    transition_model.run(parameters)
    comp_sizes = transition_model.get_outputs_df()
    
    # Get the values for the last compartment in the series
    outputs[comp_request] = comp_sizes[f"comp_{comp_request - 1}"]

# Plot
outputs_df = pd.DataFrame(outputs)
outputs_df.plot(title="Last compartment size")

## Equivalent mathematical distribution
The equivalent mathematical distribution that can capture the
delay that is implemented by including multiple compartments
in series in this way is the Erlang distribution
(a special case of the gamma distribution).
An Erlang distribution with shape parameter of one
is analogous to a single transition without intermediate
compartments.
An Erlang distribution with shape parameter of two
is analogous to having one intervening compartment between
the two compartments of interest.
An Erlang distribution with shape parameter of three
is analogous to having two intervening compartments between
the two compartments of interest, and so on.

Let's get the equivalent values to those we modelled with
our compartmental model directly from the Erlang distribution.

In [ ]:
from scipy.stats import erlang

results = {}
for shape in n_comp_requests:
    shape -= 1
    results[shape] = erlang.pdf(
        transition_model.times, 
        shape, 
        scale=parameters["sojourn_time"] / shape
    )

In [ ]:
diff_df = outputs_df.diff()
diff_df.plot(
    title="Modelled transition time"
)

In [ ]:
pd.DataFrame(results, index=transition_model.times).plot(
    title="Equivalent Erlang distributions"
)